## 📈 Predicting Premier League Final Positions Using Betting Odds & Simulation

**Competition:** English Premier League 2025/26  
**Purpose:** Estimate probabilities of final league positions using betting market information and simulation  
**Methods:** Odds-implied probabilities, Monte Carlo simulation, scenario analysis  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  

---

**Notebook first written:** `17/01/2026`  
**Last updated:** `17/01/2026`  

> This notebook develops a probabilistic framework to predict final Premier League final positions using betting odds as market-based expectations.
>
> Betting odds are transformed into implied probabilities and adjusted for bookmaker margin. These probabilities are then used to simulate the remainder of the season via Monte Carlo methods, generating distributions over final points totals and league positions.
>
> The analysis focuses on estimating the likelihood of key outcomes such as title wins, top-four finishes, relegation, and mid-table placements. Results are presented at team level with uncertainty intervals, and the framework can be extended to incorporate form, fixture difficulty, or alternative predictive inputs beyond betting markets.


In [4]:
import soccerdata as sd

[01/18/26 10:34:26] INFO     No custom team name replacements found. You can configure these in       ]8;id=546848;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=378091;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\vicky\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=179283;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=514067;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\vicky\soccerdata\config\league_dict.json.                                    

## 1. Premier League Final Standings (ESPN Scraping)
##### Using the ESPN scraper I built in my previous project.

In [7]:
import pandas as pd

year = 2025  # current Premier League season start year

url = f"https://www.espn.com/soccer/standings/_/league/ENG.1/season/{year}"
tables = pd.read_html(url)

teams_raw = tables[0]
stats = tables[1]

teams = pd.DataFrame()
teams["position"] = teams_raw.iloc[:, 0].str.extract(r"^(\d+)").astype(int)
teams["team"] = (
    teams_raw.iloc[:, 0]
    .str.replace(r"^\d+", "", regex=True)
    .str.replace(r"^[A-Z]{2,3}", "", regex=True)
    .str.strip()
)

stats.columns = ["gp", "w", "d", "l", "gf", "ga", "gd", "pts"]
stats = stats.apply(lambda c: c.astype(str)
                              .str.replace("+", "", regex=False)
                              .astype(int))

premierleague = pd.concat([teams, stats], axis=1)
# premierleague["season"] = f"{year}-{year+1}"

premierleague


,position,team,gp,w,d,l,gf,ga,gd,pts
0,1,Arsenal,22,15,5,2,40,14,26,50
1,2,Manchester City,22,13,4,5,45,21,24,43
2,3,Aston Villa,21,13,4,4,33,24,9,43
3,4,Liverpool,22,10,6,6,33,29,4,36
4,5,Manchester United,22,9,8,5,38,32,6,35
5,6,Chelsea,22,9,7,6,36,24,12,34
6,7,Brentford,22,10,3,9,35,30,5,33
7,8,Sunderland,22,8,9,5,23,23,0,33
8,9,Newcastle United,21,9,5,7,32,27,5,32
9,10,Fulham,22,9,4,9,30,31,-1,31


## 2. Get betting odds using API

In [ ]:
API_KEY = 'e97e6a2ede803956fe8d2788581f8f1f'

In [9]:
setx MY_API_KEY "e97e6a2ede803956fe8d2788581f8f1f"


SyntaxError: invalid syntax (1940107680.py, line 1)

In [ ]:
import requests
import pandas as pd

url = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"

params = {
    "apiKey": API_KEY,
    "regions": "uk",           # UK bookmakers (best for EPL)
    "markets": "h2h",          # home / draw / away
    "oddsFormat": "decimal",
    "dateFormat": "iso"
}

response = requests.get(url, params=params)
response.raise_for_status()

odds_data = response.json()
len(odds_data)
